<img style="float: center;" src='https://github.com/STScI-MIRI/MRS-ExampleNB/raw/main/assets/banner1.png' alt="stsci_logo" width="900px"/> 

<a id="title_ID"></a>
# MIRI MRS Batch Processing Notebook #

**Author**: David Law, AURA Associate Astronomer, MIRI branch
<br>
**Last Updated**: September 12, 2022
<br>
**Pipeline Version**: 1.7.0+ (unreleased)

The purpose of this notebook is to provide a framework for batch processing of MIRI MRS data through all three pipeline stages.  Data is assumed to be located in two Observation folders (science and background) according to paths set up below.

This example is set up to use observations from APT program 2732 (Stephan's Quintet ERO observations) using a standard 4-pt dither with dedicated background in all three grating settings.  Input data for this notebook can be obtained by downloading the 'uncal' files from MAST and placing them in directories set up below.  

Changes:<br>
Sep 1 2022: Add some commentary and example on how to use multicore processing in Detector 1<br>
Sep 12 2022: Disable unnecessary cube/1d spectra production for individual science exposures in Spec 2

<hr style="border:1px solid gray"> </hr>

1.<font color='white'>-</font>Configuration <a class="anchor" id="intro"></a>
------------------

In [1]:
# Set parameters to be changed here.
# It should not be necessary to edit cells below this in general unless modifying pipeline processing steps.

import sys,os, pdb

# CRDS context (if overriding)
#%env CRDS_CONTEXT jwst_0771.pmap

# Point to where the uncalibrated FITS files are from the science observation
input_dir = '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/uncal/'

# Point to where you want the output science results to go
output_dir = '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/'

# Point to where the uncalibrated FITS files are from the background observation
# If no background observation, leave this blank
input_bgdir = '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/uncal/'

# Point to where the output background observations should go
# If no background observation, leave this blank
output_bgdir = '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/'

# Whether or not to run a given pipeline stage
# Science and background are processed independently through det1+spec2, and jointly in spec3

# Science processing
dodet1=True
dospec2=True
dospec3=True

# Background processing
dodet1bg=True
dospec2bg=True

# If there is no background folder, ensure we don't try to process it
if (input_bgdir == ''):
    dodet1bg=False
    dospec2bg=False

<hr style="border:1px solid gray"> </hr>

2.<font color='white'>-</font>Imports and setup <a class="anchor" id="intro"></a>
------------------

In [2]:
# Now let's use the entire available screen width for the notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
# Basic system utilities for interacting with files
import glob
import time
import shutil
import warnings
import zipfile
import urllib.request

# Astropy utilities for opening FITS and ASCII files
from astropy.io import fits
from astropy.io import ascii
from astropy.utils.data import download_file
# Astropy utilities for making plots
from astropy.visualization import (LinearStretch, LogStretch, ImageNormalize, ZScaleInterval)

# Numpy for doing calculations
import numpy as np

# Matplotlib for making plots
import matplotlib.pyplot as plt
from matplotlib import rc

In [4]:
# Import the base JWST package
import jwst

In [5]:
# JWST pipelines (encompassing many steps)
from jwst.pipeline import Detector1Pipeline
from jwst.pipeline import Spec2Pipeline
from jwst.pipeline import Spec3Pipeline

# JWST pipeline utilities
from jwst import datamodels # JWST datamodels
from jwst.associations import asn_from_list as afl # Tools for creating association files
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase # Definition of a Lvl2 association file
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base # Definition of a Lvl3 association file

from stcal import dqflags # Utilities for working with the data quality (DQ) arrays

In [6]:
# Output subdirectories to keep science data products organized
# Note that the pipeline might complain about this as it is intended to work with everything in a single
# directory, but it nonetheless works fine for the examples given here.
det1_dir = os.path.join(output_dir, 'stage1/') # Detector1 pipeline outputs will go here
spec2_dir = os.path.join(output_dir, 'stage2/') # Spec2 pipeline outputs will go here
spec3_dir = os.path.join(output_dir, 'stage3/') # Spec3 pipeline outputs will go here

# We need to check that the desired output directories exist, and if not create them
if not os.path.exists(det1_dir):
    os.makedirs(det1_dir)
if not os.path.exists(spec2_dir):
    os.makedirs(spec2_dir)
if not os.path.exists(spec3_dir):
    os.makedirs(spec3_dir)

In [7]:
# Output subdirectories to keep background data products organized
det1_bgdir = os.path.join(output_bgdir, 'stage1/') # Detector1 pipeline outputs will go here
spec2_bgdir = os.path.join(output_bgdir, 'stage2/') # Spec2 pipeline outputs will go here

# We need to check that the desired output directories exist, and if not create them
if (output_bgdir != ''):
    if not os.path.exists(det1_bgdir):
        os.makedirs(det1_bgdir)
    if not os.path.exists(spec2_bgdir):
        os.makedirs(spec2_bgdir)

In [8]:
# Start a timer to keep track of runtime
time0 = time.perf_counter()

<hr style="border:1px solid gray"> </hr>

3.<font color='white'>-</font>Detector1 Pipeline <a class="anchor" id="det1"></a>
------------------

<div class="alert alert-block alert-warning">
In this section we process our data through the Detector1 pipeline to create Lvl2a data products (i.e., uncalibrated slope images).
See https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html
</div>

In [9]:
# First we'll define a function that will call the detector1 pipeline with our desired set of parameters
# We won't enumerate the individual steps
def rundet1(filename, outdir):
    print(filename)
    det1 = Detector1Pipeline() # Instantiate the pipeline
    det1.output_dir = outdir # Specify where the output should go
    
    # Overrides for whether or not certain steps should be skipped
    #det1.dq_init.skip = False
    #det1.saturation.skip = False
    #det1.firstframe.skip = False
    #det1.lastframe.skip = False
    #det1.reset.skip = False
    #det1.linearity.skip = False
    #det1.rscd.skip = False
    #det1.dark_current.skip = False
    #det1.refpix.skip = False
    #det1.jump.skip = False
    #det1.ramp_fit.skip = False
    #det1.gain_scale.skip = False
    
    # The jump and ramp fitting steps can benefit from multi-core processing, but this is off by default
    # Turn them on here if desired by choosing how many cores to use (quarter, half, or all)
    #det1.jump.maximum_cores='half'
    #det1.ramp_fit.maximum_cores='half'
    
    # Bad pixel mask overrides
    #det1.dq_init.override_mask = 'myfile.fits'

    # Saturation overrides
    #et1.saturation.override_saturation = 'myfile.fits'
    
    # Reset overrides
    #det1.reset.override_reset = 'myfile.fits'
        
    # Linearity overrides
    #det1.linearity.override_linearity = 'myfile.fits'

    # RSCD overrides
    #det1.rscd.override_rscd = 'myfile.fits'
        
    # DARK overrides
    #det1.dark_current.override_dark = 'myfile.fits'
        
    # GAIN overrides
    #det1.jump.override_gain = 'myfile.fits'
    #det1.ramp_fit.override_gain = 'myfile.fits'
                
    # READNOISE overrides
    #det1.jump.override_readnoise = 'myfile.fits'
    #det1.ramp_fit.override_readnoise = 'myfile.fits'
        
    det1.save_results = True # Save the final resulting _rate.fits files
    det1(filename) # Run the pipeline on an input list of files

In [10]:
# Now let's look for input files of the form *uncal.fits from the science observation
sstring = input_dir + 'jw*mirifu*uncal.fits'
lvl1b_files = sorted(glob.glob(sstring))
print('Found ' + str(len(lvl1b_files)) + ' science input files to process')

Found 24 science input files to process


In [11]:
# Run the pipeline on these input files by a simple loop over our pipeline function
if dodet1:
    for file in lvl1b_files:
        rundet1(file, det1_dir)
else:
    print('Skipping Detector1 processing')

2022-09-01 17:13:16,874 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2022-09-01 17:13:16,876 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2022-09-01 17:13:16,878 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2022-09-01 17:13:16,880 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2022-09-01 17:13:16,881 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2022-09-01 17:13:16,883 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2022-09-01 17:13:16,885 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2022-09-01 17:13:16,887 - stpipe.Detector1Pipeline.rscd - INFO - RscdStep instance created.
2022-09-01 17:13:16,888 - stpipe.Detector1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2022-09-01 17:13:16,890 - stpipe.Detector1Pipeline.lastframe - INFO - LastFrameStep instance created.
2022-09-01 1

/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/uncal/jw02732004001_02101_00001_mirifulong_uncal.fits


2022-09-01 17:13:16,977 - stpipe.Detector1Pipeline - INFO - Step Detector1Pipeline running with args ('/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/uncal/jw02732004001_02101_00001_mirifulong_uncal.fits',).
2022-09-01 17:13:16,993 - stpipe.Detector1Pipeline - INFO - Step Detector1Pipeline parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage1/', 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'save_calibrated_ramp': False, 'steps': {'group_scale': {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': ''}, 'dq_init': {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output

2022-09-01 17:13:20,537 - stpipe.Detector1Pipeline.group_scale - INFO - Step group_scale parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/uncal'}
2022-09-01 17:13:21,028 - stpipe.Detector1Pipeline.group_scale - INFO - NFRAMES and FRMDIVSR are equal; correction not needed
2022-09-01 17:13:21,029 - stpipe.Detector1Pipeline.group_scale - INFO - Step will be skipped
2022-09-01 17:13:21,034 - stpipe.Detector1Pipeline.group_scale - INFO - Step group_scale done
2022-09-01 17:13:21,180 - stpipe.Detector1Pipeline.dq_init - INFO - Step dq_init running with args (<RampModel(2, 40, 1024, 1032) from jw02732004001_02101_00001_mirifulong_uncal.fits>,).
2022-09-01 17:13:21,182 - stpipe.Detector1Pipeline.dq_init - INFO - Step dq_init paramet

2022-09-01 17:17:15,104 - stpipe.Detector1Pipeline.dark_current - INFO - Dark data nints=3, ngroups=300, nframes=1, groupgap=0
2022-09-01 17:17:16,964 - stpipe.Detector1Pipeline.dark_current - INFO - Step dark_current done
2022-09-01 17:17:17,444 - stpipe.Detector1Pipeline.refpix - INFO - Step refpix running with args (<RampModel(2, 40, 1024, 1032) from jw02732004001_02101_00001_mirifulong_uncal.fits>,).
2022-09-01 17:17:17,446 - stpipe.Detector1Pipeline.refpix - INFO - Step refpix parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/uncal', 'odd_even_columns': True, 'use_side_ref_pixels': True, 'side_smoothing_length': 11, 'side_gain': 1.0, 'odd_even_rows': True}
2022-09-01 17:17:17,659 - stpipe.Detector1Pipeline.refpix - INFO 

...

2022-09-01 21:54:13,425 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using READNOISE reference file: /grp/crds/jwst/references/jwst/jwst_miri_readnoise_0086.fits
2022-09-01 21:54:13,426 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using GAIN reference file: /grp/crds/jwst/references/jwst/jwst_miri_gain_0006.fits
2022-09-01 21:54:13,510 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using algorithm = ols
2022-09-01 21:54:13,511 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using weighting = optimal
2022-09-01 21:54:13,848 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of leading groups that are flagged as DO_NOT_USE: 1
2022-09-01 21:54:13,849 - stpipe.Detector1Pipeline.ramp_fit - INFO - MIRI dataset has all pixels in the final group flagged as DO_NOT_USE.
2022-09-01 21:57:17,805 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of groups per integration: 38
2022-09-01 21:57:17,806 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of integrations: 2
2022-09-01 21:57:18,053 - stpipe

/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/uncal/jw02732004001_02105_00004_mirifushort_uncal.fits


2022-09-01 21:57:22,299 - stpipe.Detector1Pipeline - INFO - Prefetching reference files for dataset: 'jw02732004001_02105_00004_mirifushort_uncal.fits' reftypes = ['dark', 'gain', 'ipc', 'linearity', 'mask', 'persat', 'readnoise', 'refpix', 'reset', 'rscd', 'saturation', 'superbias', 'trapdensity', 'trappars']
2022-09-01 21:57:22,438 - stpipe.Detector1Pipeline - INFO - Prefetch for DARK reference file is '/grp/crds/jwst/references/jwst/jwst_miri_dark_0081.fits'.
2022-09-01 21:57:22,447 - stpipe.Detector1Pipeline - INFO - Prefetch for GAIN reference file is '/grp/crds/jwst/references/jwst/jwst_miri_gain_0007.fits'.
2022-09-01 21:57:22,454 - stpipe.Detector1Pipeline - INFO - Prefetch for IPC reference file is '/grp/crds/jwst/references/jwst/jwst_miri_ipc_0008.fits'.
2022-09-01 21:57:22,460 - stpipe.Detector1Pipeline - INFO - Prefetch for LINEARITY reference file is '/grp/crds/jwst/references/jwst/jwst_miri_linearity_0030.fits'.
2022-09-01 21:57:22,462 - stpipe.Detector1Pipeline - INFO - 

2022-09-01 21:57:55,389 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity running with args (<RampModel(2, 40, 1024, 1032) from jw02732004001_02105_00004_mirifushort_uncal.fits>,).
2022-09-01 21:57:55,391 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/uncal'}
2022-09-01 21:57:55,429 - stpipe.Detector1Pipeline.linearity - INFO - Using Linearity reference file /grp/crds/jwst/references/jwst/jwst_miri_linearity_0030.fits
2022-09-01 21:57:58,725 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity done
2022-09-01 21:57:58,870 - stpipe.Detector1Pipeline.rscd - INFO - Step rscd running with args (<RampModel(2, 40, 1024, 1032) from jw02732004001_02105

2022-09-01 22:08:40,464 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using READNOISE reference file: /grp/crds/jwst/references/jwst/jwst_miri_readnoise_0087.fits
2022-09-01 22:08:40,464 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using GAIN reference file: /grp/crds/jwst/references/jwst/jwst_miri_gain_0007.fits
2022-09-01 22:08:40,547 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using algorithm = ols
2022-09-01 22:08:40,548 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using weighting = optimal
2022-09-01 22:08:40,883 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of leading groups that are flagged as DO_NOT_USE: 1
2022-09-01 22:08:40,885 - stpipe.Detector1Pipeline.ramp_fit - INFO - MIRI dataset has all pixels in the final group flagged as DO_NOT_USE.
2022-09-01 22:11:44,776 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of groups per integration: 38
2022-09-01 22:11:44,777 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of integrations: 2
2022-09-01 22:11:45,024 - stpipe

In [12]:
# Now let's look for input files of the form *uncal.fits from the background observation
sstring = input_bgdir + 'jw*mirifu*uncal.fits'
lvl1b_files = sorted(glob.glob(sstring))
print('Found ' + str(len(lvl1b_files)) + ' background input files to process')

Found 12 background input files to process


In [13]:
# Run the pipeline on these input files by a simple loop over our pipeline function
if dodet1bg:
    for file in lvl1b_files:
        rundet1(file, det1_bgdir)
else:
    print('Skipping Detector1 processing')

2022-09-01 22:11:46,592 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2022-09-01 22:11:46,594 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2022-09-01 22:11:46,596 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2022-09-01 22:11:46,597 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2022-09-01 22:11:46,599 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2022-09-01 22:11:46,601 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2022-09-01 22:11:46,602 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2022-09-01 22:11:46,604 - stpipe.Detector1Pipeline.rscd - INFO - RscdStep instance created.
2022-09-01 22:11:46,606 - stpipe.Detector1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2022-09-01 22:11:46,608 - stpipe.Detector1Pipeline.lastframe - INFO - LastFrameStep instance created.
2022-09-01 2

/Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/uncal/jw02732005001_02101_00001_mirifulong_uncal.fits


2022-09-01 22:11:49,105 - stpipe.Detector1Pipeline - INFO - Prefetching reference files for dataset: 'jw02732005001_02101_00001_mirifulong_uncal.fits' reftypes = ['dark', 'gain', 'ipc', 'linearity', 'mask', 'persat', 'readnoise', 'refpix', 'reset', 'rscd', 'saturation', 'superbias', 'trapdensity', 'trappars']
2022-09-01 22:11:49,295 - stpipe.Detector1Pipeline - INFO - Prefetch for DARK reference file is '/grp/crds/jwst/references/jwst/jwst_miri_dark_0083.fits'.
2022-09-01 22:11:49,312 - stpipe.Detector1Pipeline - INFO - Prefetch for GAIN reference file is '/grp/crds/jwst/references/jwst/jwst_miri_gain_0006.fits'.
2022-09-01 22:11:49,318 - stpipe.Detector1Pipeline - INFO - Prefetch for IPC reference file is '/grp/crds/jwst/references/jwst/jwst_miri_ipc_0007.fits'.
2022-09-01 22:11:49,329 - stpipe.Detector1Pipeline - INFO - Prefetch for LINEARITY reference file is '/grp/crds/jwst/references/jwst/jwst_miri_linearity_0033.fits'.
2022-09-01 22:11:49,349 - stpipe.Detector1Pipeline - INFO - P

2022-09-01 22:12:23,343 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/uncal'}
2022-09-01 22:12:23,381 - stpipe.Detector1Pipeline.linearity - INFO - Using Linearity reference file /grp/crds/jwst/references/jwst/jwst_miri_linearity_0033.fits
2022-09-01 22:12:26,434 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity done
2022-09-01 22:12:26,583 - stpipe.Detector1Pipeline.rscd - INFO - Step rscd running with args (<RampModel(2, 40, 1024, 1032) from jw02732005001_02101_00001_mirifulong_uncal.fits>,).
2022-09-01 22:12:26,585 - stpipe.Detector1Pipeline.rscd - INFO - Step rscd parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir

2022-09-01 22:21:45,505 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using READNOISE reference file: /grp/crds/jwst/references/jwst/jwst_miri_readnoise_0086.fits
2022-09-01 22:21:45,506 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using GAIN reference file: /grp/crds/jwst/references/jwst/jwst_miri_gain_0006.fits
2022-09-01 22:21:45,589 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using algorithm = ols
2022-09-01 22:21:45,589 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using weighting = optimal
2022-09-01 22:21:45,924 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of leading groups that are flagged as DO_NOT_USE: 1
2022-09-01 22:21:45,926 - stpipe.Detector1Pipeline.ramp_fit - INFO - MIRI dataset has all pixels in the final group flagged as DO_NOT_USE.
2022-09-01 22:24:47,333 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of groups per integration: 38
2022-09-01 22:24:47,334 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of integrations: 2
2022-09-01 22:24:47,575 - stpipe

...

2022-09-02 00:19:02,802 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using READNOISE reference file: /grp/crds/jwst/references/jwst/jwst_miri_readnoise_0086.fits
2022-09-02 00:19:02,803 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using GAIN reference file: /grp/crds/jwst/references/jwst/jwst_miri_gain_0006.fits
2022-09-02 00:19:02,888 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using algorithm = ols
2022-09-02 00:19:02,889 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using weighting = optimal
2022-09-02 00:19:03,222 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of leading groups that are flagged as DO_NOT_USE: 1
2022-09-02 00:19:03,224 - stpipe.Detector1Pipeline.ramp_fit - INFO - MIRI dataset has all pixels in the final group flagged as DO_NOT_USE.
2022-09-02 00:21:59,431 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of groups per integration: 38
2022-09-02 00:21:59,432 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of integrations: 2
2022-09-02 00:21:59,674 - stpipe

/Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/uncal/jw02732005001_02105_00002_mirifushort_uncal.fits


2022-09-02 00:22:03,650 - stpipe.Detector1Pipeline - INFO - Prefetching reference files for dataset: 'jw02732005001_02105_00002_mirifushort_uncal.fits' reftypes = ['dark', 'gain', 'ipc', 'linearity', 'mask', 'persat', 'readnoise', 'refpix', 'reset', 'rscd', 'saturation', 'superbias', 'trapdensity', 'trappars']
2022-09-02 00:22:03,661 - stpipe.Detector1Pipeline - INFO - Prefetch for DARK reference file is '/grp/crds/jwst/references/jwst/jwst_miri_dark_0081.fits'.
2022-09-02 00:22:03,662 - stpipe.Detector1Pipeline - INFO - Prefetch for GAIN reference file is '/grp/crds/jwst/references/jwst/jwst_miri_gain_0007.fits'.
2022-09-02 00:22:03,663 - stpipe.Detector1Pipeline - INFO - Prefetch for IPC reference file is '/grp/crds/jwst/references/jwst/jwst_miri_ipc_0008.fits'.
2022-09-02 00:22:03,664 - stpipe.Detector1Pipeline - INFO - Prefetch for LINEARITY reference file is '/grp/crds/jwst/references/jwst/jwst_miri_linearity_0030.fits'.
2022-09-02 00:22:03,666 - stpipe.Detector1Pipeline - INFO - 

2022-09-02 00:22:19,465 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity running with args (<RampModel(2, 40, 1024, 1032) from jw02732005001_02105_00002_mirifushort_uncal.fits>,).
2022-09-02 00:22:19,466 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/uncal'}
2022-09-02 00:22:19,502 - stpipe.Detector1Pipeline.linearity - INFO - Using Linearity reference file /grp/crds/jwst/references/jwst/jwst_miri_linearity_0030.fits
2022-09-02 00:22:21,542 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity done
2022-09-02 00:22:21,695 - stpipe.Detector1Pipeline.rscd - INFO - Step rscd running with args (<RampModel(2, 40, 1024, 1032) from jw02732005001_02105

2022-09-02 00:26:33,404 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using READNOISE reference file: /grp/crds/jwst/references/jwst/jwst_miri_readnoise_0087.fits
2022-09-02 00:26:33,405 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using GAIN reference file: /grp/crds/jwst/references/jwst/jwst_miri_gain_0007.fits
2022-09-02 00:26:33,487 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using algorithm = ols
2022-09-02 00:26:33,487 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using weighting = optimal
2022-09-02 00:26:33,827 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of leading groups that are flagged as DO_NOT_USE: 1
2022-09-02 00:26:33,829 - stpipe.Detector1Pipeline.ramp_fit - INFO - MIRI dataset has all pixels in the final group flagged as DO_NOT_USE.
2022-09-02 00:29:37,858 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of groups per integration: 38
2022-09-02 00:29:37,858 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of integrations: 2
2022-09-02 00:29:38,113 - stpipe

In [14]:
# Print out the time benchmark
time1 = time.perf_counter()
print(f"Runtime so far: {time1 - time0:0.4f} seconds")

Runtime so far: 26182.7007 seconds


4.<font color='white'>-</font>Spec2 Pipeline <a class="anchor" id="spec2"></a>
------------------

<div class="alert alert-block alert-warning">
In this section we process our data through the Spec2 pipeline in order to produce Lvl2b data products (i.e., calibrated slope images and quick-look data cubes and 1d spectra).  
See https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec2.html
</div>

In [15]:
# Define a function that will call the spec2 pipeline with our desired set of parameters
# We'll list the individual steps just to make it clear what's running
def runspec2(filename, outdir, nocubes=False):
    spec2 = Spec2Pipeline()
    spec2.output_dir = outdir

    # Assign_wcs overrides
    #spec2.assign_wcs.override_distortion = 'myfile.asdf'
    #spec2.assign_wcs.override_regions = 'myfile.asdf'
    #spec2.assign_wcs.override_specwcs = 'myfile.asdf'
    #spec2.assign_wcs.override_wavelengthrange = 'myfile.asdf'

    # Flatfield overrides
    #spec2.flat.override_flat = 'myfile.fits'
        
    # Straylight overrides
    #spec2.straylight.override_mrsxartcorr = 'myfile.fits'
        
    # Fringe overrides
    #spec2.fringe.override_fringe = 'myfile.fits'
    
    # Photom overrides
    #spec2.photom.override_photom = 'myfile.fits'

    # Cubepar overrides
    #spec2.cube_build.override_cubepar = 'myfile.fits'
        
    # Extract1D overrides
    #spec2.extract1d.override_extract1d = 'myfile.asdf'
    #spec2.extract1d.override_apcorr = 'myfile.asdf'
        
    # Overrides for whether or not certain steps should be skipped
    #spec2.assign_wcs.skip = False
    #spec2.bkg_subtract.skip = True
    #spec2.flat_field.skip = False
    #spec2.srctype.skip = False
    #spec2.straylight.skip = False
    #spec2.fringe.skip = False
    #spec2.photom.skip = False
    #spec2.cube_build.skip = False
    #spec2.extract_1d.skip = False
    
    # This nocubes option allows us to skip the cube building and 1d spectral extraction for individual
    # science data frames, but run it for the background data (as the 1d spectra are needed later
    # for the master background step in Spec3)
    if (nocubes):
        spec2.cube_build.skip = True
        spec2.extract_1d.skip = True
    
    # Some cube building options
    #spec2.cube_build.weighting='drizzle'
    #spec2.cube_build.coord_system='ifualign' # If aligning cubes with IFU axes instead of sky
      
    spec2.save_results = True
    spec2(filename)

In [16]:
# Look for uncalibrated science slope files from the Detector1 pipeline
sstring = det1_dir + 'jw*mirifu*rate.fits'
ratefiles = sorted(glob.glob(sstring))
ratefiles=np.array(ratefiles)
print('Found ' + str(len(ratefiles)) + ' input files to process')

Found 24 input files to process


In [17]:
if dospec2:
    for file in ratefiles:
        runspec2(file, spec2_dir, nocubes=True)
else:
    print('Skipping Spec2 processing')

2022-09-02 00:29:39,603 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.
2022-09-02 00:29:39,605 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2022-09-02 00:29:39,607 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2022-09-02 00:29:39,609 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2022-09-02 00:29:39,611 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2022-09-02 00:29:39,613 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2dStep instance created.
2022-09-02 00:29:39,617 - stpipe.Spec2Pipeline.master_background_mos - INFO - MasterBackgroundMosStep instance created.
2022-09-02 00:29:39,619 - stpipe.Spec2Pipeline.master_background_mos.flat_field - INFO - FlatFieldStep instance created.
2022-09-02 00:29:39,621 - stpipe.Spec2Pipeline.master_background_mos.pathloss - INFO - PathLossStep instance created.
2022-09-02 00:29:39,623 - stpipe.Spec2Pipeline.

2022-09-02 00:29:40,073 - stpipe.Spec2Pipeline - INFO - Prefetching reference files for dataset: 'jw02732004001_02101_00001_mirifulong_rate.fits' reftypes = ['apcorr', 'area', 'barshadow', 'camera', 'collimator', 'cubepar', 'dflat', 'disperser', 'distortion', 'drizpars', 'extract1d', 'fflat', 'filteroffset', 'flat', 'fore', 'fpa', 'fringe', 'ifufore', 'ifupost', 'ifuslicer', 'mrsxartcorr', 'msa', 'msaoper', 'ote', 'pathloss', 'photom', 'regions', 'sflat', 'speckernel', 'specprofile', 'spectrace', 'specwcs', 'wavecorr', 'wavelengthrange', 'wavemap', 'wfssbkg']
2022-09-02 00:29:40,088 - stpipe.Spec2Pipeline - INFO - Prefetch for APCORR reference file is '/grp/crds/jwst/references/jwst/jwst_miri_apcorr_0002.asdf'.
2022-09-02 00:29:40,089 - stpipe.Spec2Pipeline - INFO - Prefetch for AREA reference file is 'N/A'.
2022-09-02 00:29:40,089 - stpipe.Spec2Pipeline - INFO - Prefetch for BARSHADOW reference file is 'N/A'.
2022-09-02 00:29:40,090 - stpipe.Spec2Pipeline - INFO - Prefetch for CAMERA 

2022-09-02 00:29:46,689 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step imprint_subtract parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': True, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage1'}
2022-09-02 00:29:46,690 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step skipped.
2022-09-02 00:29:46,695 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step imprint_subtract done
2022-09-02 00:29:46,824 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging running with args (<IFUImageModel(1024, 1032) from jw02732004001_02101_00001_mirifulong_rate.fits>,).
2022-09-02 00:29:46,825 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'ou

2022-09-02 00:30:15,132 - stpipe.Spec2Pipeline.photom - INFO - Step photom done
2022-09-02 00:30:15,357 - stpipe.Spec2Pipeline.cube_build - INFO - Step cube_build running with args (<IFUImageModel(1024, 1032) from /Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage2/jw02732004001_02101_00001_mirifulong_cal.fits>,).
2022-09-02 00:30:15,359 - stpipe.Spec2Pipeline.cube_build - INFO - Step cube_build parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': True, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': False, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage1', 'channel': 'all', 'band': 'all', 'grating': 'all', 'filter': 'all', 'output_type': 'multi', 'scale1': 0.0, 'scale2': 0.0, 'scalew': 0.0, 'weighting': 'drizzle', 'coord_system': 'skyalign', 'rois': 0.0, 'roiw': 0.0, 'weight_power': 2.0, 'wavemin': None, 'wavemax': None, 's

2022-09-02 00:31:31,207 - stpipe.Spec2Pipeline.flat_field - INFO - FlatFieldStep instance created.
2022-09-02 00:31:31,208 - stpipe.Spec2Pipeline.srctype - INFO - SourceTypeStep instance created.
2022-09-02 00:31:31,210 - stpipe.Spec2Pipeline.straylight - INFO - StraylightStep instance created.
2022-09-02 00:31:31,212 - stpipe.Spec2Pipeline.fringe - INFO - FringeStep instance created.
2022-09-02 00:31:31,214 - stpipe.Spec2Pipeline.pathloss - INFO - PathLossStep instance created.
2022-09-02 00:31:31,215 - stpipe.Spec2Pipeline.barshadow - INFO - BarShadowStep instance created.
2022-09-02 00:31:31,217 - stpipe.Spec2Pipeline.wfss_contam - INFO - WfssContamStep instance created.
2022-09-02 00:31:31,219 - stpipe.Spec2Pipeline.photom - INFO - PhotomStep instance created.
2022-09-02 00:31:31,221 - stpipe.Spec2Pipeline.resample_spec - INFO - ResampleSpecStep instance created.
2022-09-02 00:31:31,224 - stpipe.Spec2Pipeline.cube_build - INFO - CubeBuildStep instance created.
2022-09-02 00:31:31,2

...

2022-09-02 01:15:45,841 - stpipe.Spec2Pipeline.photom - INFO - Step photom done
2022-09-02 01:15:46,136 - stpipe.Spec2Pipeline.cube_build - INFO - Step cube_build running with args (<IFUImageModel(1024, 1032) from /Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage2/jw02732004001_02105_00004_mirifulong_cal.fits>,).
2022-09-02 01:15:46,139 - stpipe.Spec2Pipeline.cube_build - INFO - Step cube_build parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': True, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': False, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage1', 'channel': 'all', 'band': 'all', 'grating': 'all', 'filter': 'all', 'output_type': 'multi', 'scale1': 0.0, 'scale2': 0.0, 'scalew': 0.0, 'weighting': 'drizzle', 'coord_system': 'skyalign', 'rois': 0.0, 'roiw': 0.0, 'weight_power': 2.0, 'wavemin': None, 'wavemax': None, 's

2022-09-02 01:16:40,452 - stpipe.Spec2Pipeline.straylight - INFO - StraylightStep instance created.
2022-09-02 01:16:40,454 - stpipe.Spec2Pipeline.fringe - INFO - FringeStep instance created.
2022-09-02 01:16:40,456 - stpipe.Spec2Pipeline.pathloss - INFO - PathLossStep instance created.
2022-09-02 01:16:40,457 - stpipe.Spec2Pipeline.barshadow - INFO - BarShadowStep instance created.
2022-09-02 01:16:40,459 - stpipe.Spec2Pipeline.wfss_contam - INFO - WfssContamStep instance created.
2022-09-02 01:16:40,461 - stpipe.Spec2Pipeline.photom - INFO - PhotomStep instance created.
2022-09-02 01:16:40,463 - stpipe.Spec2Pipeline.resample_spec - INFO - ResampleSpecStep instance created.
2022-09-02 01:16:40,466 - stpipe.Spec2Pipeline.cube_build - INFO - CubeBuildStep instance created.
2022-09-02 01:16:40,468 - stpipe.Spec2Pipeline.extract_1d - INFO - Extract1dStep instance created.
2022-09-02 01:16:40,688 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline running with args ('/Users/dlaw/FlightData/

2022-09-02 01:16:40,976 - stpipe.Spec2Pipeline - INFO - Prefetching reference files for dataset: 'jw02732004001_02105_00004_mirifushort_rate.fits' reftypes = ['apcorr', 'area', 'barshadow', 'camera', 'collimator', 'cubepar', 'dflat', 'disperser', 'distortion', 'drizpars', 'extract1d', 'fflat', 'filteroffset', 'flat', 'fore', 'fpa', 'fringe', 'ifufore', 'ifupost', 'ifuslicer', 'mrsxartcorr', 'msa', 'msaoper', 'ote', 'pathloss', 'photom', 'regions', 'sflat', 'speckernel', 'specprofile', 'spectrace', 'specwcs', 'wavecorr', 'wavelengthrange', 'wavemap', 'wfssbkg']
2022-09-02 01:16:40,989 - stpipe.Spec2Pipeline - INFO - Prefetch for APCORR reference file is '/grp/crds/jwst/references/jwst/jwst_miri_apcorr_0002.asdf'.
2022-09-02 01:16:40,991 - stpipe.Spec2Pipeline - INFO - Prefetch for AREA reference file is 'N/A'.
2022-09-02 01:16:40,991 - stpipe.Spec2Pipeline - INFO - Prefetch for BARSHADOW reference file is 'N/A'.
2022-09-02 01:16:40,992 - stpipe.Spec2Pipeline - INFO - Prefetch for CAMERA

2022-09-02 01:16:48,822 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step imprint_subtract parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': True, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage1'}
2022-09-02 01:16:48,823 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step skipped.
2022-09-02 01:16:48,828 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step imprint_subtract done
2022-09-02 01:16:49,005 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging running with args (<IFUImageModel(1024, 1032) from jw02732004001_02105_00004_mirifushort_rate.fits>,).
2022-09-02 01:16:49,006 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'o

2022-09-02 01:17:22,191 - stpipe.Spec2Pipeline.photom - INFO - Using area reference file: N/A
2022-09-02 01:17:24,010 - stpipe.Spec2Pipeline.photom - INFO - Using instrument: MIRI
2022-09-02 01:17:24,011 - stpipe.Spec2Pipeline.photom - INFO -  detector: MIRIFUSHORT
2022-09-02 01:17:24,012 - stpipe.Spec2Pipeline.photom - INFO -  exp_type: MIR_MRS
2022-09-02 01:17:24,012 - stpipe.Spec2Pipeline.photom - INFO -  band: LONG
2022-09-02 01:17:24,747 - stpipe.Spec2Pipeline.photom - WARNING - Keyword CDP_LOW_QUAL does not correspond to an existing DQ mnemonic, so will be ignored
2022-09-02 01:17:24,784 - stpipe.Spec2Pipeline.photom - INFO - Step photom done
2022-09-02 01:17:25,144 - stpipe.Spec2Pipeline.cube_build - INFO - Step cube_build running with args (<IFUImageModel(1024, 1032) from /Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage2/jw02732004001_02105_00004_mirifushort_cal.fits>,).
2022-09-02 01:17:25,147 - stpipe.Spec2Pipeline.cube_build - INFO - Step cube_build parameters are: {'

In [18]:
# Look for uncalibrated background slope files from the Detector1 pipeline
sstring = det1_bgdir + 'jw*mirifu*rate.fits'
ratefiles = sorted(glob.glob(sstring))
ratefiles=np.array(ratefiles)
print('Found ' + str(len(ratefiles)) + ' input files to process')

Found 12 input files to process


In [19]:
if dospec2bg:
    for file in ratefiles:
        runspec2(file, spec2_bgdir)
else:
    print('Skipping Spec2 processing')

2022-09-02 01:19:05,126 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.
2022-09-02 01:19:05,128 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2022-09-02 01:19:05,131 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2022-09-02 01:19:05,132 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2022-09-02 01:19:05,134 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2022-09-02 01:19:05,136 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2dStep instance created.
2022-09-02 01:19:05,141 - stpipe.Spec2Pipeline.master_background_mos - INFO - MasterBackgroundMosStep instance created.
2022-09-02 01:19:05,142 - stpipe.Spec2Pipeline.master_background_mos.flat_field - INFO - FlatFieldStep instance created.
2022-09-02 01:19:05,144 - stpipe.Spec2Pipeline.master_background_mos.pathloss - INFO - PathLossStep instance created.
2022-09-02 01:19:05,146 - stpipe.Spec2Pipeline.

2022-09-02 01:19:05,708 - stpipe.Spec2Pipeline - INFO - Prefetching reference files for dataset: 'jw02732005001_02101_00001_mirifulong_rate.fits' reftypes = ['apcorr', 'area', 'barshadow', 'camera', 'collimator', 'cubepar', 'dflat', 'disperser', 'distortion', 'drizpars', 'extract1d', 'fflat', 'filteroffset', 'flat', 'fore', 'fpa', 'fringe', 'ifufore', 'ifupost', 'ifuslicer', 'mrsxartcorr', 'msa', 'msaoper', 'ote', 'pathloss', 'photom', 'regions', 'sflat', 'speckernel', 'specprofile', 'spectrace', 'specwcs', 'wavecorr', 'wavelengthrange', 'wavemap', 'wfssbkg']
2022-09-02 01:19:05,724 - stpipe.Spec2Pipeline - INFO - Prefetch for APCORR reference file is '/grp/crds/jwst/references/jwst/jwst_miri_apcorr_0002.asdf'.
2022-09-02 01:19:05,725 - stpipe.Spec2Pipeline - INFO - Prefetch for AREA reference file is 'N/A'.
2022-09-02 01:19:05,726 - stpipe.Spec2Pipeline - INFO - Prefetch for BARSHADOW reference file is 'N/A'.
2022-09-02 01:19:05,727 - stpipe.Spec2Pipeline - INFO - Prefetch for CAMERA 

2022-09-02 01:19:12,483 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step imprint_subtract parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': True, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/stage1'}
2022-09-02 01:19:12,484 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step skipped.
2022-09-02 01:19:12,489 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step imprint_subtract done
2022-09-02 01:19:12,657 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging running with args (<IFUImageModel(1024, 1032) from jw02732005001_02101_00001_mirifulong_rate.fits>,).
2022-09-02 01:19:12,659 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'ou

2022-09-02 01:19:41,091 - stpipe.Spec2Pipeline.photom - INFO - Step photom done
2022-09-02 01:19:41,413 - stpipe.Spec2Pipeline.cube_build - INFO - Step cube_build running with args (<IFUImageModel(1024, 1032) from /Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/stage2/jw02732005001_02101_00001_mirifulong_cal.fits>,).
2022-09-02 01:19:41,416 - stpipe.Spec2Pipeline.cube_build - INFO - Step cube_build parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': True, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': False, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/stage1', 'channel': 'all', 'band': 'all', 'grating': 'all', 'filter': 'all', 'output_type': 'multi', 'scale1': 0.0, 'scale2': 0.0, 'scalew': 0.0, 'weighting': 'drizzle', 'coord_system': 'skyalign', 'rois': 0.0, 'roiw': 0.0, 'weight_power': 2.0, 'wavemin': None, 'wavemax': None, 's

2022-09-02 01:20:57,513 - stpipe.Spec2Pipeline.flat_field - INFO - FlatFieldStep instance created.
2022-09-02 01:20:57,515 - stpipe.Spec2Pipeline.srctype - INFO - SourceTypeStep instance created.
2022-09-02 01:20:57,517 - stpipe.Spec2Pipeline.straylight - INFO - StraylightStep instance created.
2022-09-02 01:20:57,518 - stpipe.Spec2Pipeline.fringe - INFO - FringeStep instance created.
2022-09-02 01:20:57,520 - stpipe.Spec2Pipeline.pathloss - INFO - PathLossStep instance created.
2022-09-02 01:20:57,522 - stpipe.Spec2Pipeline.barshadow - INFO - BarShadowStep instance created.
2022-09-02 01:20:57,524 - stpipe.Spec2Pipeline.wfss_contam - INFO - WfssContamStep instance created.
2022-09-02 01:20:57,525 - stpipe.Spec2Pipeline.photom - INFO - PhotomStep instance created.
2022-09-02 01:20:57,528 - stpipe.Spec2Pipeline.resample_spec - INFO - ResampleSpecStep instance created.
2022-09-02 01:20:57,530 - stpipe.Spec2Pipeline.cube_build - INFO - CubeBuildStep instance created.
2022-09-02 01:20:57,5

...

2022-09-02 01:40:32,477 - stpipe.Spec2Pipeline.photom - INFO - Step photom done
2022-09-02 01:40:32,780 - stpipe.Spec2Pipeline.cube_build - INFO - Step cube_build running with args (<IFUImageModel(1024, 1032) from /Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/stage2/jw02732005001_02105_00002_mirifulong_cal.fits>,).
2022-09-02 01:40:32,783 - stpipe.Spec2Pipeline.cube_build - INFO - Step cube_build parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': True, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': False, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/stage1', 'channel': 'all', 'band': 'all', 'grating': 'all', 'filter': 'all', 'output_type': 'multi', 'scale1': 0.0, 'scale2': 0.0, 'scalew': 0.0, 'weighting': 'drizzle', 'coord_system': 'skyalign', 'rois': 0.0, 'roiw': 0.0, 'weight_power': 2.0, 'wavemin': None, 'wavemax': None, 's

2022-09-02 01:41:26,550 - stpipe.Spec2Pipeline.straylight - INFO - StraylightStep instance created.
2022-09-02 01:41:26,552 - stpipe.Spec2Pipeline.fringe - INFO - FringeStep instance created.
2022-09-02 01:41:26,554 - stpipe.Spec2Pipeline.pathloss - INFO - PathLossStep instance created.
2022-09-02 01:41:26,555 - stpipe.Spec2Pipeline.barshadow - INFO - BarShadowStep instance created.
2022-09-02 01:41:26,557 - stpipe.Spec2Pipeline.wfss_contam - INFO - WfssContamStep instance created.
2022-09-02 01:41:26,559 - stpipe.Spec2Pipeline.photom - INFO - PhotomStep instance created.
2022-09-02 01:41:26,562 - stpipe.Spec2Pipeline.resample_spec - INFO - ResampleSpecStep instance created.
2022-09-02 01:41:26,564 - stpipe.Spec2Pipeline.cube_build - INFO - CubeBuildStep instance created.
2022-09-02 01:41:26,566 - stpipe.Spec2Pipeline.extract_1d - INFO - Extract1dStep instance created.
2022-09-02 01:41:26,792 - stpipe.Spec2Pipeline - INFO - Step Spec2Pipeline running with args ('/Users/dlaw/FlightData/

2022-09-02 01:41:27,061 - stpipe.Spec2Pipeline - INFO - Prefetching reference files for dataset: 'jw02732005001_02105_00002_mirifushort_rate.fits' reftypes = ['apcorr', 'area', 'barshadow', 'camera', 'collimator', 'cubepar', 'dflat', 'disperser', 'distortion', 'drizpars', 'extract1d', 'fflat', 'filteroffset', 'flat', 'fore', 'fpa', 'fringe', 'ifufore', 'ifupost', 'ifuslicer', 'mrsxartcorr', 'msa', 'msaoper', 'ote', 'pathloss', 'photom', 'regions', 'sflat', 'speckernel', 'specprofile', 'spectrace', 'specwcs', 'wavecorr', 'wavelengthrange', 'wavemap', 'wfssbkg']
2022-09-02 01:41:27,076 - stpipe.Spec2Pipeline - INFO - Prefetch for APCORR reference file is '/grp/crds/jwst/references/jwst/jwst_miri_apcorr_0002.asdf'.
2022-09-02 01:41:27,077 - stpipe.Spec2Pipeline - INFO - Prefetch for AREA reference file is 'N/A'.
2022-09-02 01:41:27,078 - stpipe.Spec2Pipeline - INFO - Prefetch for BARSHADOW reference file is 'N/A'.
2022-09-02 01:41:27,078 - stpipe.Spec2Pipeline - INFO - Prefetch for CAMERA

2022-09-02 01:41:34,941 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step imprint_subtract parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': True, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/stage1'}
2022-09-02 01:41:34,942 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step skipped.
2022-09-02 01:41:34,946 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step imprint_subtract done
2022-09-02 01:41:35,125 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging running with args (<IFUImageModel(1024, 1032) from jw02732005001_02105_00002_mirifushort_rate.fits>,).
2022-09-02 01:41:35,127 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'o

2022-09-02 01:42:08,435 - stpipe.Spec2Pipeline.photom - INFO - Using area reference file: N/A
2022-09-02 01:42:10,286 - stpipe.Spec2Pipeline.photom - INFO - Using instrument: MIRI
2022-09-02 01:42:10,287 - stpipe.Spec2Pipeline.photom - INFO -  detector: MIRIFUSHORT
2022-09-02 01:42:10,287 - stpipe.Spec2Pipeline.photom - INFO -  exp_type: MIR_MRS
2022-09-02 01:42:10,288 - stpipe.Spec2Pipeline.photom - INFO -  band: LONG
2022-09-02 01:42:11,110 - stpipe.Spec2Pipeline.photom - WARNING - Keyword CDP_LOW_QUAL does not correspond to an existing DQ mnemonic, so will be ignored
2022-09-02 01:42:11,146 - stpipe.Spec2Pipeline.photom - INFO - Step photom done
2022-09-02 01:42:11,509 - stpipe.Spec2Pipeline.cube_build - INFO - Step cube_build running with args (<IFUImageModel(1024, 1032) from /Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/stage2/jw02732005001_02105_00002_mirifushort_cal.fits>,).
2022-09-02 01:42:11,512 - stpipe.Spec2Pipeline.cube_build - INFO - Step cube_build parameters are: {'

In [20]:
# Print out the time benchmark
time1 = time.perf_counter()
print(f"Runtime so far: {time1 - time0:0.4f} seconds")

Runtime so far: 30633.7173 seconds


5.<font color='white'>-</font>Spec3 Pipeline: Default configuration (4 per-channel cubes)<a class="anchor" id="spec3"></a>
------------------

<div class="alert alert-block alert-warning">
Here we'll run the Spec3 pipeline to produce a composite data cube from all dithered exposures.
We will need to create an association file from all science and background data in order for the pipeline to use them appropriately.

A word of caution: the data cubes created by the JWST pipeline are in SURFACE BRIGHTNESS units (MJy/steradian), not flux units.  What that means is that if you intend to sum spectra within an aperture you need to be sure to multiply by the pixel area in steradians first in order to get a spectrum in flux units (the PIXAR_SR keyword can be found in the SCI extension header).  This correction is already build into the pipeline Extract1D algorithm.

See https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec3.html
    
</div>

In [21]:
# Define a useful function to write out a Lvl3 association file from an input list
# Note that any background exposures have to be of type x1d.
def writel3asn(scifiles, bgfiles, asnfile, prodname):
    # Define the basic association of science files
    asn = afl.asn_from_list(scifiles, rule=DMS_Level3_Base, product_name=prodname)
        
    # Add background files to the association
    nbg=len(bgfiles)
    for ii in range(0,nbg):
        asn['products'][0]['members'].append({'expname': bgfiles[ii], 'exptype': 'background'})
        
    # Write the association to a json file
    _, serialized = asn.dump()
    with open(asnfile, 'w') as outfile:
        outfile.write(serialized)

In [22]:
# Find and sort all of the input files

# Science Files need the cal.fits files
sstring = spec2_dir + 'jw*mirifu*cal.fits'
calfiles = np.array(sorted(glob.glob(sstring)))

# Background Files need the x1d.fits files
sstring = spec2_bgdir + 'jw*mirifu*x1d.fits'
bgfiles = np.array(sorted(glob.glob(sstring)))

print('Found ' + str(len(calfiles)) + ' science files to process')
print('Found ' + str(len(bgfiles)) + ' background files to process')

Found 24 science files to process
Found 12 background files to process


In [23]:
# Make an association file that includes all of the different exposures
asnfile=os.path.join(output_dir, 'l3asn.json')
if dospec3:
    writel3asn(calfiles, bgfiles, asnfile, 'Level3')

In [24]:
# Define a function that will call the spec3 pipeline with our desired set of parameters
# This is designed to run on an association file
def runspec3(filename):
    # This initial setup is just to make sure that we get the latest parameter reference files
    # pulled in for our files.  This is a temporary workaround to get around an issue with
    # how this pipeline calling method works.
    crds_config = Spec3Pipeline.get_config_from_reference(filename)
    spec3 = Spec3Pipeline.from_config_section(crds_config)
    
    spec3.output_dir = spec3_dir
    spec3.save_results = True
    
    # Cube building configuration options
    # spec3.cube_build.output_file = 'bandcube' # Custom output name
    # spec3.cube_build.output_type = 'band' # 'band', 'channel', or 'multi' type cube output
    # spec3.cube_build.channel = '1' # Build everything from just channel 1 into a single cube (we could also choose '2','3','4', or 'ALL')
    # spec3.cube_build.weighting = 'drizzle' # 'emsm' or 'drizzle'
    # spec3.cube_build.coord_system = 'ifualign' # 'ifualign', 'skyalign', or 'internal_cal'
    # spec3.cube_build.scale1 = 0.5 # Output cube spaxel scale (arcsec) in dimension 1 if setting it by hand
    # spec3.cube_build.scale2 = 0.5 # Output cube spaxel scale (arcsec) in dimension 2 if setting it by hand
    # spec3.cube_build.scalew = 0.002 # Output cube spaxel size (microns) in dimension 3 if setting it by hand
    
    # Overrides for whether or not certain steps should be skipped
    #spec3.assign_mtwcs.skip = False
    #spec3.master_background.skip = True
    #spec3.outlier_detection.skip = True
    #spec3.mrs_imatch.skip = True
    #spec3.cube_build.skip = False
    #spec3.extract_1d.skip = False
    
    # Cubepar overrides
    #spec3.cube_build.override_cubepar = 'myfile.fits'
        
    # Extract1D overrides
    #spec3.extract1d.override_extract1d = 'myfile.asdf'
    #spec3.extract1d.override_apcorr = 'myfile.asdf'

    spec3(filename)

In [25]:
if dospec3:
    runspec3(asnfile)
else:
    print('Skipping Spec3 processing')

2022-09-02 01:43:52,734 - stpipe - INFO - PARS-OUTLIERDETECTIONSTEP parameters found: /grp/crds/jwst/references/jwst/jwst_miri_pars-outlierdetectionstep_0051.asdf
2022-09-02 01:43:52,764 - stpipe - INFO - PARS-SPEC3PIPELINE parameters found: /grp/crds/jwst/references/jwst/jwst_miri_pars-spec3pipeline_0002.asdf
2022-09-02 01:43:52,792 - stpipe.Spec3Pipeline - INFO - Spec3Pipeline instance created.
2022-09-02 01:43:52,794 - stpipe.Spec3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2022-09-02 01:43:52,795 - stpipe.Spec3Pipeline.master_background - INFO - MasterBackgroundStep instance created.
2022-09-02 01:43:52,797 - stpipe.Spec3Pipeline.mrs_imatch - INFO - MRSIMatchStep instance created.
2022-09-02 01:43:52,799 - stpipe.Spec3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2022-09-02 01:43:52,802 - stpipe.Spec3Pipeline.resample_spec - INFO - ResampleSpecStep instance created.
2022-09-02 01:43:52,804 - stpipe.Spec3Pipeline.cube_build - INFO - 

2022-09-02 01:45:37,518 - stpipe.Spec3Pipeline.master_background - INFO - Accumulating data from input spectrum 2
2022-09-02 01:45:37,552 - stpipe.Spec3Pipeline.master_background - INFO - Accumulating data from input spectrum 3
2022-09-02 01:45:37,574 - stpipe.Spec3Pipeline.master_background - INFO - Accumulating data from input spectrum 4
2022-09-02 01:45:37,607 - stpipe.Spec3Pipeline.master_background - INFO - Accumulating data from input spectrum 5
2022-09-02 01:45:37,626 - stpipe.Spec3Pipeline.master_background - INFO - Accumulating data from input spectrum 6
2022-09-02 01:45:37,658 - stpipe.Spec3Pipeline.master_background - INFO - Accumulating data from input spectrum 7
2022-09-02 01:45:37,678 - stpipe.Spec3Pipeline.master_background - INFO - Accumulating data from input spectrum 8
2022-09-02 01:45:37,710 - stpipe.Spec3Pipeline.master_background - INFO - Accumulating data from input spectrum 9
2022-09-02 01:45:37,725 - stpipe.Spec3Pipeline.master_background - INFO - Accumulating d

2022-09-02 01:48:28,609 - stpipe.Spec3Pipeline.outlier_detection - INFO - Blotting back to jw02732004001_02101_00003_mirifulong_cal.fits
2022-09-02 01:48:30,955 - stpipe.Spec3Pipeline.outlier_detection - INFO - Blotting back to jw02732004001_02101_00004_mirifulong_cal.fits
2022-09-02 01:48:32,471 - stpipe.CubeBuildStep - INFO - CubeBuildStep instance created.
2022-09-02 01:48:32,472 - stpipe.CubeBuildStep - INFO - Starting IFU Cube Building Step
2022-09-02 01:48:32,473 - stpipe.CubeBuildStep - INFO - Input interpolation: pointcloud
2022-09-02 01:48:32,474 - stpipe.CubeBuildStep - INFO - Coordinate system to use: skyalign
2022-09-02 01:48:32,474 - stpipe.CubeBuildStep - INFO - Weighting method for point cloud: emsm
2022-09-02 01:48:32,475 - stpipe.CubeBuildStep - INFO - Power weighting distance: 2.0
2022-09-02 01:48:32,475 - stpipe.CubeBuildStep - INFO - Cube Type: Single cubes
2022-09-02 01:48:32,836 - stpipe.Spec3Pipeline.outlier_detection - INFO - The desired cubes cover the MIRI Cha

2022-09-02 01:49:06,128 - stpipe.Spec3Pipeline.outlier_detection - INFO - Cube covers channel, subchannel: 2, medium
2022-09-02 01:49:06,476 - stpipe.Spec3Pipeline.outlier_detection - INFO - # of detector pixels not mapped to output plane: 1743 with wavelength below 8.666000076453201
2022-09-02 01:49:06,477 - stpipe.Spec3Pipeline.outlier_detection - INFO - # of detector pixels not mapped to output plane: 1227 with wavelength above 10.154000146430917
2022-09-02 01:49:07,425 - stpipe.Spec3Pipeline.outlier_detection - INFO - Number of spectral tear planes adjusted: 0
2022-09-02 01:49:10,526 - stpipe.Spec3Pipeline.outlier_detection - INFO - Subchannel listing: ['medium']
2022-09-02 01:49:10,977 - stpipe.Spec3Pipeline.outlier_detection - INFO - # of detector pixels not mapped to output plane: 1743 with wavelength below 8.666000076453201
2022-09-02 01:49:10,978 - stpipe.Spec3Pipeline.outlier_detection - INFO - # of detector pixels not mapped to output plane: 1227 with wavelength above 10.154

2022-09-02 01:49:57,965 - stpipe.Spec3Pipeline.outlier_detection - INFO - Subchannel listing: ['short']
2022-09-02 01:49:58,460 - stpipe.Spec3Pipeline.outlier_detection - INFO - # of detector pixels not mapped to output plane: 2721 with wavelength below 4.89800009544706
2022-09-02 01:49:58,461 - stpipe.Spec3Pipeline.outlier_detection - INFO - # of detector pixels not mapped to output plane: 3469 with wavelength above 5.742000135185663
2022-09-02 01:49:59,655 - stpipe.Spec3Pipeline.outlier_detection - INFO - Number of spectral tear planes adjusted: 0
2022-09-02 01:50:02,804 - stpipe.Spec3Pipeline.outlier_detection - INFO - Subchannel listing: ['short']
2022-09-02 01:50:03,317 - stpipe.Spec3Pipeline.outlier_detection - INFO - # of detector pixels not mapped to output plane: 2721 with wavelength below 4.89800009544706
2022-09-02 01:50:03,318 - stpipe.Spec3Pipeline.outlier_detection - INFO - # of detector pixels not mapped to output plane: 3469 with wavelength above 5.742000135185663
2022-

...

2022-09-02 01:55:25,341 - stpipe.CubeBuildStep - INFO - CubeBuildStep instance created.
2022-09-02 01:55:25,342 - stpipe.CubeBuildStep - INFO - Starting IFU Cube Building Step
2022-09-02 01:55:25,343 - stpipe.CubeBuildStep - INFO - Input interpolation: pointcloud
2022-09-02 01:55:25,343 - stpipe.CubeBuildStep - INFO - Coordinate system to use: skyalign
2022-09-02 01:55:25,344 - stpipe.CubeBuildStep - INFO - Weighting method for point cloud: emsm
2022-09-02 01:55:25,345 - stpipe.CubeBuildStep - INFO - Power weighting distance: 2.0
2022-09-02 01:55:25,346 - stpipe.CubeBuildStep - INFO - Cube Type: Single cubes
2022-09-02 01:55:25,737 - stpipe.Spec3Pipeline.outlier_detection - INFO - The desired cubes cover the MIRI Channels: ['2']
2022-09-02 01:55:25,738 - stpipe.Spec3Pipeline.outlier_detection - INFO - The desired cubes cover the MIRI subchannels: ['long']
2022-09-02 01:55:25,738 - stpipe.Spec3Pipeline.outlier_detection - INFO - Reading cube parameter file /grp/crds/jwst/references/jwst

2022-09-02 01:56:14,326 - stpipe.Spec3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 4379 (0.41%)
2022-09-02 01:56:14,434 - stpipe.Spec3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 1847 (0.17%)
2022-09-02 01:56:14,542 - stpipe.Spec3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 4001 (0.38%)
2022-09-02 01:56:14,649 - stpipe.Spec3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 856 (0.08%)
2022-09-02 01:56:14,757 - stpipe.Spec3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 4245 (0.40%)
2022-09-02 01:56:14,864 - stpipe.Spec3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 1144 (0.11%)
2022-09-02 01:56:14,974 - stpipe.Spec3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 4368 (0.41%)
2022-09-02 01:56:15,079 - stpipe.Spec3Pipeline.outlier_detection - INFO - New pixels flagged as outliers: 1079 (0.10%)
2022-09-02 01:56:15,183 - stpipe.Spec3Pipeline.ou

2022-09-02 01:57:08,266 - stpipe.Spec3Pipeline.cube_build - INFO - Mapping all pixels to output to determine IFU foot print
2022-09-02 01:57:08,589 - stpipe.Spec3Pipeline.cube_build - INFO - Mapping all pixels to output to determine IFU foot print
2022-09-02 01:57:08,937 - stpipe.Spec3Pipeline.cube_build - INFO - Mapping all pixels to output to determine IFU foot print
2022-09-02 01:57:09,244 - stpipe.Spec3Pipeline.cube_build - INFO - Mapping all pixels to output to determine IFU foot print
2022-09-02 01:57:09,568 - stpipe.Spec3Pipeline.cube_build - INFO - Mapping all pixels to output to determine IFU foot print
2022-09-02 01:57:09,879 - stpipe.Spec3Pipeline.cube_build - INFO - Mapping all pixels to output to determine IFU foot print
2022-09-02 01:57:10,185 - stpipe.Spec3Pipeline.cube_build - INFO - Mapping all pixels to output to determine IFU foot print
2022-09-02 01:57:10,488 - stpipe.Spec3Pipeline.cube_build - INFO - Mapping all pixels to output to determine IFU foot print
2022-09-

2022-09-02 02:00:51,758 - stpipe.Spec3Pipeline.cube_build - INFO - # of detector pixels not mapped to output plane: 184 with wavelength above 18.02559392947137
2022-09-02 02:00:59,018 - stpipe.Spec3Pipeline.cube_build - INFO - Average # of holes/wavelength plane is < 1
2022-09-02 02:00:59,019 - stpipe.Spec3Pipeline.cube_build - INFO - Total # of holes for IFU cube is : 0
2022-09-02 02:00:59,047 - stpipe.Spec3Pipeline.cube_build - INFO - Number of spectral tear planes adjusted: 0
2022-09-02 02:01:02,005 - stpipe.Spec3Pipeline.cube_build - INFO - Cube Geometry:
2022-09-02 02:01:02,005 - stpipe.Spec3Pipeline.cube_build - INFO - axis#  Naxis  CRPIX    CRVAL      CDELT(arcsec)  Min & Max (xi, eta arcsec)
2022-09-02 02:01:02,006 - stpipe.Spec3Pipeline.cube_build - INFO - Axis 1    33  17.00 -20.98487458   0.34999999  -5.77499990   5.77499990
2022-09-02 02:01:02,007 - stpipe.Spec3Pipeline.cube_build - INFO - Axis 2    33  17.00  33.97595497   0.34999999  -5.77499990   5.77499990
2022-09-02 02

2022-09-02 02:04:01,698 - stpipe.Spec3Pipeline.extract_1d - INFO - Step extract_1d parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': 'x1d', 'search_output_file': False, 'input_dir': '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04', 'smoothing_length': None, 'bkg_fit': None, 'bkg_order': None, 'bkg_sigma_clip': 3.0, 'log_increment': 50, 'subtract_background': None, 'use_source_posn': None, 'center_xy': None, 'apply_apcorr': True, 'soss_threshold': 0.01, 'soss_n_os': 2, 'soss_transform': None, 'soss_tikfac': None, 'soss_width': 40.0, 'soss_bad_pix': 'model', 'soss_modelname': None}
2022-09-02 02:04:01,739 - stpipe.Spec3Pipeline.extract_1d - INFO - Using EXTRACT1D reference file /grp/crds/jwst/references/jwst/jwst_miri_extract1d_0003.asdf
2022-09-02 02:04:01,757 - stpipe.Spec3Pipeline.extract_1d - INFO - Using APCORR file 

In [26]:
# Print out the time benchmark
time1 = time.perf_counter()
print(f"Runtime so far: {time1 - time0:0.4f} seconds")

Runtime so far: 32091.7095 seconds


<hr style="border:1px solid gray"> </hr>

<img style="float: center;" src="https://www.stsci.edu/~dlaw/stsci_logo.png" alt="stsci_logo" width="200px"/> 